# Geometry Tutorial

This is an introductory walkthrough of the symbolic [geometry package](../api/symforce.geo.html) in symforce. Symforce natively contains the following geometry objects: complex numbers, quaternions, dual quaternions, matrices, 2D and 3D rotations, and 2D and 3D poses (translation + rotation).

In this tutorial we will demonstrate a few of the ways these geometry objects can be constructed and composed. Here we pay special attention to 3D rotations, poses, and vectors, as they are typically the most commonly used geometric objects.

In [ ]:
# Setup
import symforce
symforce.set_backend('sympy')
symforce.set_log_level('warning')

from symforce.notebook_util import display
from symforce import sympy as sm
from symforce import geo
from symforce import ops

## Rotations

Rotations can be defined from and converted to a number of different representations as shown below, but always use a quaternion as the underlying representation. We use the notation world_R_body to represent a rotation that rotates a point written in the body frame into the world frame.

Note that in some cases a small epsilon can be provided to prevent numerical errors (e.g. division by zero) when converting to/from rotation representations. Furthermore, converting between certain representations can require additional symbolic expressions to guard against degenerate cases. For example, a rotation constructed from a rotation matrix results in more complexity than when constructed using an axis-angle representation as shown below.

In [ ]:
# Identity definition
display(geo.Rot3())

In [ ]:
# Symbolic definition
display(geo.Rot3.symbolic('R'))

In [ ]:
# To/From rotation matrix

# Rotate about x-axis
theta = sm.Symbol('theta')
R_mat = geo.Matrix([[1, 0, 0], [0, sm.cos(theta), -sm.sin(theta)], [0, sm.sin(theta), sm.cos(theta)]])
R = geo.Rot3.from_rotation_matrix(R_mat)

display(R_mat)
display(R)  # Note the additional expressions required to avoid numerical errors
display(R.to_rotation_matrix())

In [ ]:
# To/From Euler angles
R = geo.Rot3.from_yaw_pitch_roll(0, 0, theta) # Yaw rotation only
ypr = R.to_yaw_pitch_roll()

display(R)
display(ops.StorageOps.simplify(list(ypr)))  # Simplify YPR expression

In [ ]:
# From axis-angle representation

# Rotate about x-axis
R = geo.Rot3.from_angle_axis(
    angle=theta,
    axis=geo.Vector3(1, 0, 0)
)

display(R)

Now that we can construct rotations, we can use them to rotate vectors as one would expect.

In [ ]:
world_R_body = geo.Rot3.symbolic('R')  # Rotation defining orientation of body frame wrt world frame
body_t_point = geo.Vector3.symbolic('p')  # Point written in body frame
world_t_point = world_R_body * body_t_point  # Point written in world frame
display(world_t_point)

Chaining rotations and inverting rotations works as one would expect as well:

In [ ]:
body_R_cam = geo.Rot3.symbolic('R_cam')
world_R_cam = world_R_body * body_R_cam

# Rotation inverse = negate vector part of quaternion
cam_R_body = body_R_cam.inverse()
display(body_R_cam)
display(cam_R_body)

We can also easily substitute numerical values into symbolic expressions using geo objects themselves. This makes it very convenient to substitute numeric values into large symbolic expressions constructed using many different geo objects.

In [ ]:
world_R_body_numeric = geo.Rot3.from_yaw_pitch_roll(0.1, -2.3, 0.7)
display(world_t_point.subs(world_R_body, world_R_body_numeric))

## Poses

Poses are defined as a rotation plus a translation, and are constructed as such. We use the notation world_T_body to represent a pose that transforms from the body frame to the world frame.

In [ ]:
# Symbolic construction
world_T_body = geo.Pose3.symbolic('T')
display(world_T_body)

In [ ]:
# Construction from a rotation and translation
world_R_body = geo.Rot3.symbolic('R')  # Orientation of body frame wrt world frame
world_t_body = geo.Vector3.symbolic('t')  # Position of body frame wrt world frame written in the world frame
world_T_body = geo.Pose3(
    R=world_R_body,
    t=world_t_body
)
display(world_T_body)

Similar to rotations, we can compose poses with poses, compose poses with 3D points, and invert poses as one would expect.

In [ ]:
# Compose pose with a pose
body_T_cam = geo.Pose3.symbolic('T_cam')
world_T_cam = world_T_body * body_T_cam

# Compose pose with a point
body_t_point = geo.Vector3.symbolic('p')  # Position relative to body frame written in body frame
display(world_T_body * body_t_point)  # Equivalent to: world_R_body * body_t_point + world_t_body

In [ ]:
# Invert a pose
body_T_world = world_T_body.inverse()
display(world_T_body)
display(body_T_world)

## Vectors and matricies

Vectors and matrices are all represented using subclasses of geo.Matrix class, and can be constrcuted in several different ways as shown below.

In [ ]:
# Matrix construction. The statements below all create the same 2x3 matrix object

# Construction from 2D list
m1 = geo.Matrix([[1, 2, 3], [4, 5, 6]])

# Construction using specified size + data
m2 = geo.Matrix(2, 3, [1, 2, 3, 4, 5, 6])

# geo.MatrixNM creates a matrix with shape NxM (defined by default 6x6 matrices and smaller)
m3 = geo.Matrix23(1, 2, 3, 4, 5, 6)
m4 = geo.Matrix23([1, 2, 3, 4, 5, 6])

# Construction using aliases
m5 = geo.M([[1, 2, 3], [4, 5, 6]])
m6 = geo.M(2, 3, [1, 2, 3, 4, 5, 6])
m7 = geo.M23(1, 2, 3, 4, 5, 6)
m8 = geo.M23([1, 2, 3, 4, 5, 6])

# Construction from block matrices of appropriate dimensions
m9 = geo.Matrix23.block_matrix([[geo.M13([1, 2, 3])], [geo.M13([3, 4, 5])]])

In [ ]:
# Vector constructors. The statements below all create the same 3x1 vector object

# Construction from 2D list
v1 = geo.Matrix([[1], [2], [3]])

# Construction from 1D list. We assume a 1D list represents a column vector.
v2 = geo.Matrix([1, 2, 3])

# Construction using aliases (defined by default for 9x1 vectors and smaller)
v3 = geo.Matrix31(1, 2, 3)
v4 = geo.M31(1, 2, 3)
v5 = geo.Vector3(1, 2, 3)
v6 = geo.V3(1, 2, 3)

We can also use a few typical matrix constructors:

In [ ]:
# Matrix of zeros
z1 = geo.Matrix23.zero()
z2 = geo.Matrix.zeros(2, 3)

# Matrix of ones
o1 = geo.Matrix23.one()
o2 = geo.Matrix.ones(2, 3)

Note that in symforce we define matrices as groups under element-wise addition, meaning that the "identity" element of a given sized matrix is the zero matrix.

In [ ]:
zero_matrix = geo.Matrix33.identity()
identity_matrix = geo.Matrix33.matrix_identity()  # We could also write geo.Matrix.eye(3, 3)

display(zero_matrix)
display(identity_matrix)

And, of course, matrix math works as one would expect:

In [ ]:
# Matrix multiplication
m23 = geo.M23.symbolic('lhs')
m31 = geo.V3.symbolic('rhs')
display(m23 * m31)

In [ ]:
# Vector operations
norm = m31.norm()
squared_norm = m31.squared_norm()
unit_vec = m31.normalized()
display(unit_vec)

In [ ]:
m33 = 5 * geo.Matrix33.matrix_identity()  # Element-wise multiplication with scalar
display(m33.matrix_inverse())  # Matrix inverse

One of the most powerful operations we can use matrices for is to compute jacobians with respect to other geo objects. By default we compute jacobians with respect to the tangent space of the given object.

In [ ]:
R0 = geo.Rot3.symbolic("R0")
R1 = geo.Rot3.symbolic("R1")
residual = geo.M(R0.local_coordinates(R1))
display(residual)

In [ ]:
jacobian = residual.jacobian(R1)
# The jacobian is quite a complex symbolic expression, so we don't display it for convenience
# The shape is equal to (dimension of residual) x (dimension of tangent space)
display(jacobian.shape)

## General properties of geo objects

### Storage operations

All geometric types implement the "Storage" interface. This means that they can:

1. Be serialized into a list of scalar expressions (`.to_storage()`)
2. Be reconstructed from a list of scalar expressions (`.from_storage()`)
3. Use common symbolic operations (symbolic construction, substitution, simplification, etc.)

In [ ]:
# Serialization to scalar list
rot = geo.Rot3()
elements = rot.to_storage()
assert len(elements) == rot.storage_dim()
display(elements)

In [ ]:
# Construction from scalar list
rot2 = geo.Rot3.from_storage(elements)
assert rot == rot2

In [ ]:
# Symbolic operations
rot_sym = geo.Rot3.symbolic("rot_sym")
rot_num = rot_sym.subs(rot_sym, rot)

display(rot_sym)
display(rot_num)
display(rot_num.simplify())  # Simplify interal symbolic expressions
display(rot_num.evalf())  # Numerical evaluation

### Group operations

All geometric types also implement the "Group" interface, meaning that geometric objects:

1. Can be composed with objects of the same type to produce an object of the same type (`.compose()`)
2. Have an identity element (`.identity()`)
3. Can be inverted (`.inverse()`)
4. Can be created to represent the relation between two other objects of the same type (`.between()`)

In [ ]:
# Construct two random rotations
R1 = geo.Rot3.random()
R2 = geo.Rot3.random()

# Composition
display(R1.compose(R2))  # For rotations this is the same as R1 * R2

In [ ]:
# Identity
R_identity = geo.Rot3.identity()
display(R1)
display(R_identity * R1)

In [ ]:
# Inverse
R1_inv = R1.inverse()
display(R_identity)
display(R1_inv * R1)

In [ ]:
# Between
R_delta = R1.between(R2)
display(R1 * R_delta)
display(R2)

### Lie Group operations

Rotations, poses, and matrices all implement the "LieGroup" interface, meaning that they each have a tangent space. Objects which are a Lie Group can:

1. Be used to compute the tangent space vector about the identity element (`.to_tangent()`)
2. Be constructed from a tangent space vector about the identity element (`.from_tangent()`)
3. Be perturbed by a tangent space vector about the given element (`.retract()`)
4. Be used to compute the tangent space perturbation needed to obtain another given element (`.local_coordinates()`)
5. Be used to compute a jacobian describing the relation between the underlying data of the object (e.g. a quaternion for a rotation) and the tangent space vector about the given element (`.storage_D_tangent()`)

In [ ]:
# To/From tangent space vector about identity element
R1 = geo.Rot3.random()
tangent_vec = R1.to_tangent()
R1_recovered = geo.Rot3.from_tangent(tangent_vec)

assert len(tangent_vec) == R1.tangent_dim()
display(R1)
display(R1_recovered)

In [ ]:
# Tangent space perturbations
R2 = R1.retract([0.1, 2.3, -0.5])  # Perturb R1 by the given tangent space vector (relative to R1)
recovered_tangent_vec = R1.local_coordinates(R2)

display(recovered_tangent_vec)

In [ ]:
# Jacobian of storage w.r.t tangent space perturbation

# We chain storage_D_tangent together with jacobians of larger symbolic expressions taken
# with respect to the symbolic elements of the object (e.g. a quaternion for rotations) to compute
# the jacobian wrt the tanget space about the element.
# I.e. residual_D_tangent = residual_D_storage * storage_D_tangent

jacobian = R1.storage_D_tangent()
assert jacobian.shape == (R1.storage_dim(), R1.tangent_dim())

For more detials on Storage/Group/LieGroup operations, see the [Concept tutorial](../notebooks/ops_tutorial.html).